<div id="top">
<!-- <div style="background-image: url(https://www.emtmadrid.es/getattachment/da3be644-cb9d-44db-8011-e3f40f1c5c34); opacity: 0.2"/> -->
<img src="https://www.gmv.com/sites/default/files/content/image/2021/11/03/115/gmv_rgbredblack.png" alt="GMV Logo" style="width: 200px">
<img src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/CEI/LOGOTIPO%20leyenda%20color%20JPG%20p.png" alt="UPM Logo" style="float: right; width: 200px">
<h1><b>QA: AEMET dataset 🌥️</b></h1>
<h5 style="text-align: right">INESDATA-MOV</h5>
</div>

# Análisis de calidad
Este cuaderno analiza la calidad del dataset proveniente de la fuente de datos de la Agencia Estatal de Meteorología ([AEMET](https://www.aemet.es/es/portada)). La calidad del mismo se validará teniendo en cuenta los siguientes aspectos:

* Análisis del dataset
* Análisis de las variables
* Conversiones de tipos de datos

La **calidad del dato** se refiere a la medida en que los datos son adecuados para su uso, por lo que es esencial para garantizar la confiabilidad y utilidad de los datos en diversas aplicaciones y contextos. Así, en este notebook se evaluarán también las cinco dimensiones de la calidad del dato:
1. **Unicidad**: Ausencia de duplicados o registros repetidos en un conjunto de datos. Los datos son únicos cuando cada registro o entidad en el conjunto de datos es único y no hay duplicados presentes.
2. **Exactitud**: Los datos exactos son libres de errores y representan con precisión la realidad que están destinados a describir. Esto implica que los datos deben ser correctos y confiables para su uso en análisis y toma de decisiones.
3. **Completitud**: Los datos completos contienen toda la información necesaria para el análisis y no tienen valores faltantes o nulos que puedan afectar la interpretación o validez de los resultados.
4. **Consistencia**: Los datos consistentes mantienen el mismo formato, estructura y significado en todas las instancias, lo que facilita su comparación y análisis sin ambigüedad.
5. **Validez**: Medida en que los datos son precisos y representan con exactitud la realidad que están destinados a describir. 

<div class="admonition info">
<p class="admonition-title">Nota</p>
<p>
Este dataset ha sido creado ejecutando el comando <code>create</code> del paquete de Python <a href="https://github.com/oeg-upm/inesdata-mov-data-generation"><code>inesdata_mov_datasets</code></a>.<br>
Para poder ejecutar este comando es necesario haber ejecutado antes el comando <code>extract</code>, que realiza la extracción de datos de la API de la AEMET y los almacena en Minio. El comando <code>create</code> se encargaría de descargar dichos datos y unirlos todos en un único dataset.
</p>
</div>

In [1]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
ROOT_PATH = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
DATA_PATH = os.path.join(ROOT_PATH, "data", "processed")
AEMET_DATA_PATH = os.path.join(DATA_PATH, "aemet")

'/home/pphs/data-generation/data/processed/aemet'

**Cada fila de este dataset representa la meteorología de Madrid para una determinada fecha y hora concretos.**

<div class="admonition warning">
<p class="admonition-title">-</p>
<p>
Vamos a analizar la calidad del dataset generado para los días comprendidos entre 02/03/2024 - 31/03/2024 que son los días de los que disponemos información actualmente.
</p>
</div>

In [3]:
#####################################################################
# Creacción del dataset completo con toda la información disponible #
#####################################################################

# Directorio con los CSV
directory = AEMET_DATA_PATH

aemet_data = []

for root, directories, files in os.walk(directory):
    for filename in files:
        if filename.endswith('.csv'):
            filepath = os.path.join(root, filename)
            aemet_data.append(pd.read_csv(filepath))

aemet_dataset = pd.concat(aemet_data, ignore_index=True)

## Análisis del dataset

<h10> El dataset anterior recoge datos de casi todo marzo del 2024. En este apartado se analiza si el dataset presenta alguna anomalía de manera general. </h10>

In [4]:
# Visualización del dataset
aemet_dataset.head()

,estadoCielo_value,estadoCielo_descripcion,precipitacion_value,probPrecipitacion_value,probTormenta_value,nieve_value,probNieve_value,temperatura_value,sensTermica_value,humedadRelativa_value,direccion,velocidad,vientoAndRachaMax_value,datetime
0,12n,Poco nuboso,0,NaN,NaN,0,NaN,8.0,5.0,77.0,['SO'],['16'],NaN,2024-03-02 00:00:00
1,12n,Poco nuboso,0,NaN,NaN,0,NaN,8.0,5.0,77.0,NaN,NaN,27.0,2024-03-02 00:00:00
2,15n,Muy nuboso,0,NaN,NaN,0,NaN,8.0,5.0,80.0,['O'],['17'],NaN,2024-03-02 01:00:00
3,15n,Muy nuboso,0,NaN,NaN,0,NaN,8.0,5.0,80.0,NaN,NaN,27.0,2024-03-02 01:00:00
4,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-02 01:07:00


<h10> Se observa que para la misma fecha existen varios datos. Esto se debe a que en los datos de la AEMET la variable *vientoAndRachaMax* aparece separada del resto. Lo que haremos entonces será hacer una agrupación por fecha para que los datos que nos falten de *vientoAndRachaMax* se añadan al resto. </h10>

<h10> Veamos un ejemplo de esto para una fecha dada: 2024-03-02 00:00:00 </h10>

In [5]:
aemet_dataset[aemet_dataset["datetime"]=="2024-03-02 00:00:00"]

,estadoCielo_value,estadoCielo_descripcion,precipitacion_value,probPrecipitacion_value,probTormenta_value,nieve_value,probNieve_value,temperatura_value,sensTermica_value,humedadRelativa_value,direccion,velocidad,vientoAndRachaMax_value,datetime
0,12n,Poco nuboso,0,NaN,NaN,0,NaN,8.0,5.0,77.0,['SO'],['16'],NaN,2024-03-02 00:00:00
1,12n,Poco nuboso,0,NaN,NaN,0,NaN,8.0,5.0,77.0,NaN,NaN,27.0,2024-03-02 00:00:00
1626,12n,Poco nuboso,0.0,NaN,NaN,0,NaN,8.0,5.0,77.0,['SO'],['16'],NaN,2024-03-02 00:00:00
1627,12n,Poco nuboso,0.0,NaN,NaN,0,NaN,8.0,5.0,77.0,NaN,NaN,27.0,2024-03-02 00:00:00


<h10> Para esta fecha existen hasta 4 registros diferentes. De estos 4 registros, 2 de ellos son duplicados de los otros dos, así que además de disponer información incompleta se tiene información duplicada. Vemos a continuación si ocurre lo mismo para el resto de fechas: </h10>

In [6]:
aemet_dataset.groupby(["datetime"])["datetime"].count().sort_values(ascending=False)

datetime
2024-03-02 00:00:00    4
2024-03-21 13:00:00    4
2024-03-21 05:00:00    4
2024-03-21 06:00:00    4
2024-03-21 07:00:00    4
                      ..
2024-03-07 13:19:00    2
2024-03-19 07:13:00    2
2024-03-27 19:01:00    2
2024-03-05 07:13:00    2
2024-03-18 13:19:00    2
Name: datetime, Length: 773, dtype: int64

<h10> Para eliminar la información duplicada cada variable debe tener el mismo tipo de datos para todas sus filas. Nos fijaremos en los tipos actuales para establecer el tipo de cada variable. </h10>

In [7]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2848 entries, 0 to 2847
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   estadoCielo_value        2624 non-null   object 
 1   estadoCielo_descripcion  2624 non-null   object 
 2   precipitacion_value      2624 non-null   object 
 3   probPrecipitacion_value  224 non-null    float64
 4   probTormenta_value       224 non-null    float64
 5   nieve_value              2624 non-null   object 
 6   probNieve_value          224 non-null    float64
 7   temperatura_value        2604 non-null   float64
 8   sensTermica_value        2604 non-null   float64
 9   humedadRelativa_value    2604 non-null   float64
 10  direccion                1302 non-null   object 
 11  velocidad                1302 non-null   object 
 12  vientoAndRachaMax_value  1302 non-null   float64
 13  datetime                 2848 non-null   object 
dtypes: float64(7), object(7)

<h10> Variables como *precipitacion_value* y *nieve_value* están declaradas como strings cuando deberían ser variables float. Si vemos los diferentes valores que poseen, se detecta que existe el valor "Ip" que es el que hace que estas variables se determinen como strings. Dado que "Ip" significa precipitación inapreciable o cantidad de nieve inapreciable se sustituirán estos valores por 0.0 </h10>

In [8]:
aemet_dataset["precipitacion_value"].unique()

array(['0', nan, '0.1', '0.6', '6', '0.2', '1', '0.5', '4', '0.9', '0.8',
       '0.7', 'Ip', '0.3', '3', '2', '0.4', 0.0, 0.1, 0.6, 6.0, 0.2, 1.0,
       0.5, 4.0, 0.9, 3.0, 2.0, 0.4, 0.3, 0.7, 0.8], dtype=object)

In [9]:
aemet_dataset["nieve_value"].unique()

array(['0', nan, 'Ip', 0.0], dtype=object)

In [10]:
aemet_dataset = aemet_dataset.replace({'precipitacion_value': {"Ip":0.0}, 'nieve_value': {"Ip":0.0}})

In [11]:
aemet_dataset["precipitacion_value"].unique()

array(['0', nan, '0.1', '0.6', '6', '0.2', '1', '0.5', '4', '0.9', '0.8',
       '0.7', 0.0, '0.3', '3', '2', '0.4', 0.1, 0.6, 6.0, 0.2, 1.0, 0.5,
       4.0, 0.9, 3.0, 2.0, 0.4, 0.3, 0.7, 0.8], dtype=object)

In [12]:
aemet_dataset["precipitacion_value"].unique()

array(['0', nan, '0.1', '0.6', '6', '0.2', '1', '0.5', '4', '0.9', '0.8',
       '0.7', 0.0, '0.3', '3', '2', '0.4', 0.1, 0.6, 6.0, 0.2, 1.0, 0.5,
       4.0, 0.9, 3.0, 2.0, 0.4, 0.3, 0.7, 0.8], dtype=object)

<h10> De esta manera ya se puede seleccionar un tipo para cada variable: </h10>

In [13]:
aemet_dataset = aemet_dataset.astype({'estadoCielo_value': 'str', 'estadoCielo_descripcion':'str',\
                      'precipitacion_value': 'float64', 'probPrecipitacion_value': 'float64',\
                      'probTormenta_value': 'float64', 'nieve_value': 'float64',\
                      'probNieve_value': 'float64', 'temperatura_value': 'float64',\
                      'sensTermica_value': 'float64', 'humedadRelativa_value': 'float64',\
                      'vientoAndRachaMax_value': 'float64'                    
                      })

In [14]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2848 entries, 0 to 2847
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   estadoCielo_value        2848 non-null   object 
 1   estadoCielo_descripcion  2848 non-null   object 
 2   precipitacion_value      2624 non-null   float64
 3   probPrecipitacion_value  224 non-null    float64
 4   probTormenta_value       224 non-null    float64
 5   nieve_value              2624 non-null   float64
 6   probNieve_value          224 non-null    float64
 7   temperatura_value        2604 non-null   float64
 8   sensTermica_value        2604 non-null   float64
 9   humedadRelativa_value    2604 non-null   float64
 10  direccion                1302 non-null   object 
 11  velocidad                1302 non-null   object 
 12  vientoAndRachaMax_value  1302 non-null   float64
 13  datetime                 2848 non-null   object 
dtypes: float64(9), object(5)

<h10> En este punto se pueden eliminar los duplicados: </h10>

In [15]:
aemet_dataset = aemet_dataset.drop_duplicates()

In [16]:
aemet_dataset.groupby(["datetime"])["datetime"].count().sort_values(ascending=False)

datetime
2024-03-02 00:00:00    2
2024-03-21 13:00:00    2
2024-03-21 05:00:00    2
2024-03-21 06:00:00    2
2024-03-21 07:00:00    2
                      ..
2024-03-07 13:19:00    1
2024-03-19 07:13:00    1
2024-03-27 19:01:00    1
2024-03-05 07:13:00    1
2024-03-18 13:19:00    1
Name: datetime, Length: 773, dtype: int64

<h10> También se puede ya resumir la información de dos filas en una única fila:</h10>

In [17]:
aemet_dataset[aemet_dataset["datetime"]=="2024-03-02 00:00:00"]

,estadoCielo_value,estadoCielo_descripcion,precipitacion_value,probPrecipitacion_value,probTormenta_value,nieve_value,probNieve_value,temperatura_value,sensTermica_value,humedadRelativa_value,direccion,velocidad,vientoAndRachaMax_value,datetime
0,12n,Poco nuboso,0.0,NaN,NaN,0.0,NaN,8.0,5.0,77.0,['SO'],['16'],NaN,2024-03-02 00:00:00
1,12n,Poco nuboso,0.0,NaN,NaN,0.0,NaN,8.0,5.0,77.0,NaN,NaN,27.0,2024-03-02 00:00:00


In [18]:
aemet_reduced = aemet_dataset.groupby(["datetime"])[['direccion','velocidad','vientoAndRachaMax_value']].sum()
aemet_dataset = aemet_dataset.drop(columns=["direccion", "velocidad", "vientoAndRachaMax_value"])

aemet_dataset = aemet_dataset.merge(aemet_reduced, how="inner", on= "datetime").drop_duplicates()

In [19]:
aemet_dataset.groupby(["datetime"])["datetime"].count().sort_values(ascending=False)

datetime
2024-03-02 00:00:00    1
2024-03-21 07:00:00    1
2024-03-22 06:00:00    1
2024-03-22 07:00:00    1
2024-03-22 07:13:00    1
                      ..
2024-03-13 07:00:00    1
2024-03-13 07:13:00    1
2024-03-13 08:00:00    1
2024-03-13 09:00:00    1
2024-03-31 23:00:00    1
Name: datetime, Length: 773, dtype: int64

<h10> Terminamos comprobando que el dataset del que disponemos en este punto tiene sentido en términos de cantidad de registros: </h10>

In [20]:
aemet_dataset_test = aemet_dataset.copy()

aemet_dataset_test["datetime"] = pd.to_datetime(aemet_dataset_test["datetime"])

aemet_dataset_test["day"] = aemet_dataset_test["datetime"].dt.day
aemet_dataset_test["month"] = aemet_dataset_test["datetime"].dt.month
aemet_dataset_test["year"] = aemet_dataset_test["datetime"].dt.year
aemet_dataset_test["hour"] = aemet_dataset_test["datetime"].dt.hour
aemet_dataset_test["minute"] = aemet_dataset_test["datetime"].dt.minute

aemet_dataset_test[aemet_dataset_test["minute"]==0].groupby(["month", "day"])[["estadoCielo_value"]].count()

estadoCielo_value
month day                   
3     2                   24
      3                   24
      4                   24
      5                   24
      6                   24
      7                   24
      8                   24
      11                  24
      12                  24
      13                  24
      14                  24
      15                  24
      16                  24
      17                  24
      18                  24
      19                  24
      20                  24
      21                  24
      22                  23
      23                  23
      24                  23
      25                  23
      26                  23
      27                  23
      28                  23
      29                  23
      30                  23
      31                  22

In [21]:
aemet_dataset.reset_index(drop=True, inplace=True)

<h10> De los resultados obtenidos se observa que el dataset ya presenta el número de registros que se supone (1 por hora o aproximadamente 1 por hora). Si agrupásemos por horas y minutos existen casos en los que no se dispone sólo información horaria sino que existen varios valores para la misma hora. Esto lo veremos más adelante ya que corresponde más bien con el análisis de variables. </h10>

## Análisis de las variables y conversiones de tipos

<h10> Realizamos conjúntamente el análisis de las variables y las conversiones a su tipo determinado de manera conjunta ya que ambas cosas van ligadas. En pasos anteriores se ha realizado ya la conversión de tipos pero de manera sencilla, en este paso estudiamos más cautelosamente cada caso.</h10>

In [22]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   estadoCielo_value        773 non-null    object 
 1   estadoCielo_descripcion  773 non-null    object 
 2   precipitacion_value      661 non-null    float64
 3   probPrecipitacion_value  112 non-null    float64
 4   probTormenta_value       112 non-null    float64
 5   nieve_value              661 non-null    float64
 6   probNieve_value          112 non-null    float64
 7   temperatura_value        651 non-null    float64
 8   sensTermica_value        651 non-null    float64
 9   humedadRelativa_value    651 non-null    float64
 10  datetime                 773 non-null    object 
 11  direccion                773 non-null    object 
 12  velocidad                773 non-null    object 
 13  vientoAndRachaMax_value  773 non-null    float64
dtypes: float64(9), object(5)
m

### Estado cielo

<h10> El estado del cielo lo describen dos variables: *estadoCielo_value* y *estadoCielo_descripcion*. Conociendo la correspondencia entre ambas variables se podría prescindir de una de ellas.</h10>

In [23]:
aemet_dataset[["estadoCielo_value", "estadoCielo_descripcion"]].groupby(["estadoCielo_descripcion", "estadoCielo_value"]).count()

Empty DataFrame
Columns: []
Index: [(Bruma, 82), (Bruma, 82n), (Cubierto, 16), (Cubierto, 16n), (Cubierto con lluvia, 26), (Cubierto con lluvia, 26n), (Cubierto con lluvia escasa, 46), (Cubierto con lluvia escasa, 46n), (Cubierto con nieve, 36), (Cubierto con tormenta, 54), (Cubierto con tormenta, 54n), (Cubierto con tormenta y lluvia escasa, 64), (Cubierto con tormenta y lluvia escasa, 64n), (Despejado, 11), (Despejado, 11n), (Intervalos nubosos, 13), (Intervalos nubosos con lluvia escasa, 43), (Intervalos nubosos con lluvia escasa, 43n), (Muy nuboso, 15), (Muy nuboso, 15n), (Muy nuboso con lluvia escasa, 45), (Muy nuboso con lluvia escasa, 45n), (Muy nuboso con tormenta y lluvia escasa, 63n), (Niebla , 81), (Nubes altas, 17), (Nubes altas, 17n), (Nuboso, 14), (Nuboso, 14n), (Nuboso con lluvia escasa, 44), (Nuboso con tormenta y lluvia escasa, 62), (Poco nuboso, 12), (Poco nuboso, 12n), (nan, nan)]

<h10> En la tabla anterior se observa que, a pesar de que por cada valor de descripción existan uno o varios valores, estos tienen el mismo significado. Por ello eliminaremos la columna *estadoCielo_descripcion* y haremos que los valores de *estadoCielo_value* sean valores numéricos para un mejor uso de esta variable en el modelo de predicción. Finalmente renombraremos esta variable a *estado_cielo*</h10>

In [24]:
# Cambiamos los valores 12n, 17n, etc. por 12, 17, etc. ya que representan el 
# mismo valor de estadoCielo
aemet_dataset["estadoCielo_value"] = aemet_dataset["estadoCielo_value"].replace(to_replace=r'(\d+)n', value=r'\1', regex=True).replace('nan', np.nan)

In [25]:
aemet_dataset[["estadoCielo_value", "estadoCielo_descripcion"]].groupby(["estadoCielo_descripcion", "estadoCielo_value"]).count()

,
estadoCielo_descripcion,estadoCielo_value
Bruma,82
Cubierto,16
Cubierto con lluvia,26
Cubierto con lluvia escasa,46
Cubierto con nieve,36
Cubierto con tormenta,54
Cubierto con tormenta y lluvia escasa,64
Despejado,11
Intervalos nubosos,13


In [26]:
# Eliminamos la columna estadoCielo_descripcion
aemet_dataset.drop(columns='estadoCielo_descripcion', inplace=True)

In [27]:
# Cambiamos el tipo de la variable estadoCielo_value a numérico y 
# cambiamos el nombre de esta columna por estado_cielo
aemet_dataset['estadoCielo_value'] = pd.to_numeric(aemet_dataset['estadoCielo_value'], errors='coerce')
aemet_dataset.rename(columns={'estadoCielo_value': 'estado_cielo'}, inplace=True)

In [28]:
aemet_dataset["estado_cielo"].value_counts(dropna=False)

estado_cielo
17.0    169
11.0    163
NaN     112
16.0    105
12.0     61
46.0     47
64.0     47
15.0     27
43.0     15
14.0     10
36.0      3
54.0      3
45.0      2
26.0      2
82.0      2
63.0      1
44.0      1
81.0      1
62.0      1
13.0      1
Name: count, dtype: int64

### Precipitación

<h10> La precipitación la describen tres variables: *precipitacion_value*, *probPrecipitacion_value* y *probTormenta_value*. Veamos si todas ellas son necesarias para el análisis.</h10>

In [29]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   estado_cielo             661 non-null    float64
 1   precipitacion_value      661 non-null    float64
 2   probPrecipitacion_value  112 non-null    float64
 3   probTormenta_value       112 non-null    float64
 4   nieve_value              661 non-null    float64
 5   probNieve_value          112 non-null    float64
 6   temperatura_value        651 non-null    float64
 7   sensTermica_value        651 non-null    float64
 8   humedadRelativa_value    651 non-null    float64
 9   datetime                 773 non-null    object 
 10  direccion                773 non-null    object 
 11  velocidad                773 non-null    object 
 12  vientoAndRachaMax_value  773 non-null    float64
dtypes: float64(10), object(3)
memory usage: 78.6+ KB


<h10> Se observa que tanto *probPrecipitacion_value* como *probTormenta_value* sólo tienen valores diferentes a los nulos para el 15% de la muestra total. Si se observa el dataset detenidamente, nos damos cuenta de que estas probabilidades no están establecidas a horas determinadas (00:00:00, 01:00:00, etc.) sino a horas del tipo 00:07:00. Esto se debe a que las probabilidades se corresponden con periodos de tiempo. En verdad 00:07:00 indica que esa probabilidad se prevee entre las 00:00:00 y las 07:00:00. Vamos a observar si se pueden juntar estos datos para horas determinadas. </h10>

In [30]:
aemet_dataset[~aemet_dataset["probPrecipitacion_value"].isnull()][["datetime", "probPrecipitacion_value"]].head()

,datetime,probPrecipitacion_value
2,2024-03-02 01:07:00,0.0
9,2024-03-02 07:13:00,100.0
16,2024-03-02 13:19:00,100.0
23,2024-03-02 19:01:00,70.0
30,2024-03-03 01:07:00,0.0


In [31]:
aemet_dataset_prob = aemet_dataset.copy()

aemet_dataset_prob["datetime"] = pd.to_datetime(aemet_dataset_prob["datetime"])

aemet_dataset_prob["day"] = aemet_dataset_prob["datetime"].dt.day
aemet_dataset_prob["month"] = aemet_dataset_prob["datetime"].dt.month
aemet_dataset_prob["year"] = aemet_dataset_prob["datetime"].dt.year
aemet_dataset_prob["hour"] = aemet_dataset_prob["datetime"].dt.hour
aemet_dataset_prob["minute"] = aemet_dataset_prob["datetime"].dt.minute
aemet_dataset_prob["second"] = aemet_dataset_prob["datetime"].dt.second

In [32]:
aemet_dataset_prob = aemet_dataset_prob[["day", "month", "hour", "minute", "second", "probPrecipitacion_value", "probTormenta_value", "probNieve_value"]]

In [33]:
aemet_dataset_prob[0:10]

,day,month,hour,minute,second,probPrecipitacion_value,probTormenta_value,probNieve_value
0,2,3,0,0,0,NaN,NaN,NaN
1,2,3,1,0,0,NaN,NaN,NaN
2,2,3,1,7,0,0.0,0.0,0.0
3,2,3,2,0,0,NaN,NaN,NaN
4,2,3,3,0,0,NaN,NaN,NaN
5,2,3,4,0,0,NaN,NaN,NaN
6,2,3,5,0,0,NaN,NaN,NaN
7,2,3,6,0,0,NaN,NaN,NaN
8,2,3,7,0,0,NaN,NaN,NaN
9,2,3,7,13,0,100.0,25.0,0.0


In [34]:
aemet_dataset_prob[0:10].fillna(method="ffill")

,day,month,hour,minute,second,probPrecipitacion_value,probTormenta_value,probNieve_value
0,2,3,0,0,0,NaN,NaN,NaN
1,2,3,1,0,0,NaN,NaN,NaN
2,2,3,1,7,0,0.0,0.0,0.0
3,2,3,2,0,0,0.0,0.0,0.0
4,2,3,3,0,0,0.0,0.0,0.0
5,2,3,4,0,0,0.0,0.0,0.0
6,2,3,5,0,0,0.0,0.0,0.0
7,2,3,6,0,0,0.0,0.0,0.0
8,2,3,7,0,0,0.0,0.0,0.0
9,2,3,7,13,0,100.0,25.0,0.0


<h10> Como se observa en las tablas anteriores *ffill* es un buen método para conseguir los datos que faltan. El siguiente paso sería eliminar los registros con *minute!=0*. Aprovecharemos ya también para dividir datetime en día, mes, hora, minutos y segundos. </h10>

In [35]:
aemet_dataset["datetime"] = pd.to_datetime(aemet_dataset["datetime"])

aemet_dataset["day"] = aemet_dataset["datetime"].dt.day
aemet_dataset["month"] = aemet_dataset["datetime"].dt.month
aemet_dataset["year"] = aemet_dataset["datetime"].dt.year
aemet_dataset["hour"] = aemet_dataset["datetime"].dt.hour
aemet_dataset["minute"] = aemet_dataset["datetime"].dt.minute
aemet_dataset["second"] = aemet_dataset["datetime"].dt.second

aemet_dataset.drop(columns=["datetime"], inplace=True)

In [36]:
aemet_dataset.fillna(method="ffill", inplace=True)

In [37]:
aemet_dataset = aemet_dataset[aemet_dataset.minute==0]

In [38]:
aemet_dataset.reset_index(drop=True, inplace=True)

In [39]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   estado_cielo             661 non-null    float64
 1   precipitacion_value      661 non-null    float64
 2   probPrecipitacion_value  659 non-null    float64
 3   probTormenta_value       659 non-null    float64
 4   nieve_value              661 non-null    float64
 5   probNieve_value          659 non-null    float64
 6   temperatura_value        661 non-null    float64
 7   sensTermica_value        661 non-null    float64
 8   humedadRelativa_value    661 non-null    float64
 9   direccion                661 non-null    object 
 10  velocidad                661 non-null    object 
 11  vientoAndRachaMax_value  661 non-null    float64
 12  day                      661 non-null    int32  
 13  month                    661 non-null    int32  
 14  year                     6

### Nieve

<h10> Algo similar a lo que ocurría con la precipitación ocurre con la nieve. Se analizará mejor la *probNieve_value* en el EDA.</h10>

### Dirección

La variable *direccion* representa los valores de norte, sur, este, oeste. Como vienen en una lista, los sacamos y los convertimos a categorías.

In [40]:
aemet_dataset["direccion"].value_counts()

direccion
['SO']    143
['NE']    135
['O']     117
['S']     104
['SE']     54
['E']      48
['NO']     37
['N']      13
0          10
Name: count, dtype: int64

In [41]:
aemet_dataset["direccion"] = aemet_dataset["direccion"].str.replace("['", "").str.replace("']", "").astype('category')
aemet_dataset["direccion"].unique()

['SO', 'O', 'NO', 'NE', 'N', 'S', 'SE', 'E', NaN]
Categories (8, object): ['E', 'N', 'NE', 'NO', 'O', 'S', 'SE', 'SO']

### Velocidad

Los valores de la variable *velocidad* vienen en una lista, los sacamos y los convertimos a *float*.

In [42]:
aemet_dataset['velocidad'].value_counts().head()

velocidad
['10']    54
['8']     47
['9']     46
['7']     39
['6']     34
Name: count, dtype: int64

In [43]:
aemet_dataset["velocidad"] = aemet_dataset["velocidad"].str.replace("['", "").str.replace("']", "").astype(float)
aemet_dataset["velocidad"].value_counts().head()

velocidad
10.0    54
8.0     47
9.0     46
7.0     39
6.0     34
Name: count, dtype: int64

In [44]:
aemet_dataset[0:20]

,estado_cielo,precipitacion_value,probPrecipitacion_value,probTormenta_value,nieve_value,probNieve_value,temperatura_value,sensTermica_value,humedadRelativa_value,direccion,velocidad,vientoAndRachaMax_value,day,month,year,hour,minute,second
0,12.0,0.0,NaN,NaN,0.0,NaN,8.0,5.0,77.0,SO,16.0,27.0,2,3,2024,0,0,0
1,15.0,0.0,NaN,NaN,0.0,NaN,8.0,5.0,80.0,O,17.0,27.0,2,3,2024,1,0,0
2,15.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,82.0,O,17.0,33.0,2,3,2024,2,0,0
3,16.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,84.0,SO,20.0,40.0,2,3,2024,3,0,0
4,16.0,0.0,0.0,0.0,0.0,0.0,8.0,4.0,86.0,O,25.0,35.0,2,3,2024,4,0,0
5,16.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,84.0,SO,21.0,37.0,2,3,2024,5,0,0
6,16.0,0.0,0.0,0.0,0.0,0.0,8.0,5.0,86.0,SO,23.0,44.0,2,3,2024,6,0,0
7,16.0,0.0,0.0,0.0,0.0,0.0,8.0,4.0,82.0,SO,27.0,51.0,2,3,2024,7,0,0
8,16.0,0.0,100.0,25.0,0.0,0.0,8.0,4.0,81.0,SO,32.0,47.0,2,3,2024,8,0,0
9,46.0,0.1,100.0,25.0,0.0,0.0,8.0,4.0,80.0,SO,28.0,47.0,2,3,2024,9,0,0


In [45]:
aemet_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   estado_cielo             661 non-null    float64 
 1   precipitacion_value      661 non-null    float64 
 2   probPrecipitacion_value  659 non-null    float64 
 3   probTormenta_value       659 non-null    float64 
 4   nieve_value              661 non-null    float64 
 5   probNieve_value          659 non-null    float64 
 6   temperatura_value        661 non-null    float64 
 7   sensTermica_value        661 non-null    float64 
 8   humedadRelativa_value    661 non-null    float64 
 9   direccion                651 non-null    category
 10  velocidad                651 non-null    float64 
 11  vientoAndRachaMax_value  661 non-null    float64 
 12  day                      661 non-null    int32   
 13  month                    661 non-null    int32   
 14  year      